# TP 3 : Dynamic programming applied to the knapsack problem and the shortest path

### Initialisation (à faire une seule fois)

In [17]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


### Récupération des données

In [16]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Problème du sac à dos


In [19]:
function calculate_max_value(item_values, item_weights, item_index, current_capacity, dp_table)
    # Base case: no capacity in knapsack
    if current_capacity == 0
        return 0
    # If item's weight is more than current capacity, skip the item
    elseif item_weights[item_index] > current_capacity
        return dp_table[item_index - 1, current_capacity]
    else
        # Calculate value with and without the current item, choose the maximum
        value_without_item = dp_table[item_index - 1, current_capacity]
        value_with_item = dp_table[item_index - 1, current_capacity - item_weights[item_index]] + item_values[item_index]
        return max(value_without_item, value_with_item)
    end
end

function knapsack(item_values, item_weights, max_capacity)
    num_items = length(item_values)
    # Initialize a 2D matrix with zeros
    dp_table = fill(0, (0:num_items, 0:max_capacity))

    # Populate the dynamic programming table
    for item_index in 1:num_items
        for current_capacity in 0:max_capacity
            dp_table[item_index, current_capacity] = calculate_max_value(item_values, item_weights, item_index, current_capacity, dp_table)
        end
    end

    # Retrieve the maximum value that can be achieved with the given capacity
    return dp_table[num_items, max_capacity]
end


knapsack (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [13]:
function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)
    fct_obj = knapsack(price, weight, capacity)
    return fct_obj
end

SolveKnapInstance (generic function with 1 method)

In [6]:
# Valeurs des objets
values = [60, 100, 120]
# Poids des objets
weights = [10, 20, 30]
# Capacité du sac à dos
capacity = 50

knapsack(values, weights, capacity)

220

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [20]:
result1 = SolveKnapInstance("./InstancesKnapSack/test.opb.txt")
println("\n******\n\nOptimal value = ",result1)
# Fonction pour lire les données d'un fichier
function read_knapsack_data(filename)
    # Vous devez implémenter cette fonction pour lire les données de chaque fichier
    # et renvoyer item_values, item_weights, et max_capacity
end

# Liste des fichiers à tester
files = ["./InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_2_-1901.opb.txt", "./InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_3_-1944.opb.txt", "./InstancesKnapSack/strongly_correlated/knapPI_3_100_1000_4_-3773.opb.txt"]

# Boucle pour tester chaque fichier
for file in files
    item_values, item_weights, max_capacity = readKnaptxtInstance(file)
    result = knapsack(item_values, item_weights, max_capacity)
    println("Résultat pour $file: $result")
end


******

Optimal value = 65
Résultat pour ./InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_2_-1901.opb.txt: 2497
Résultat pour ./InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_3_-1944.opb.txt: 2772
Résultat pour ./InstancesKnapSack/strongly_correlated/knapPI_3_100_1000_4_-3773.opb.txt: 5673


In [22]:
files = ["InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_2_-1901.opb.txt", "InstancesKnapSack/almost_strongly_correlated/knapPI_5_50_1000_3_-1944.opb.txt", "InstancesKnapSack/strongly_correlated/knapPI_3_100_1000_4_-3773.opb.txt"]

for file in files
    debut = time()
    BestProfit, BestSolution = SolveKnapInstance(file)
    fin = time()
    temps = fin - debut
    println("Résultat pour $file: $BestProfit")
    println("Temps pour $file: $temps")
    println("Solution pour $file: $BestSolution")
end

LoadError: BoundsError: attempt to access Int64 at index [2]

### Affichage du résultat final

In [22]:
function solveNdisplayKnap(filename)

    println("\n Branch-and-Bound for solving a knapsack problem. \n\n Solving instance '" * filename * "'\n")

    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = solveKnapInstance(filename)

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    println("\n Branch-and-bound tree visualization : start display ...")
    display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
    println("... end display. \n\n")

end

solveNdisplayKnap (generic function with 1 method)

# Bonus

In [5]:
function initializeDistances(graph, source)
    n = size(graph, 1)
    distances = fill(Inf, n)
    distances[convert(Int, source)] = 0
    return distances
end

function calculatePredecessors(graph)
    n = size(graph, 1)
    predecs = [Int[] for _ in 1:n]  # Crée un tableau de tableaux vides pour chaque sommet

    for i in 1:n
        for j in 1:n
            if graph[i, j] != 0  # Vérifie s'il existe une arête du sommet i au sommet j
                push!(predecs[j], i)  # Ajoute i comme prédécesseur de j
            end
        end
    end

    return predecs
end


function bellmanFord(graph, source)
    n = size(graph, 1)
    distances = initializeDistances(graph, source)
    predecs = calculatePredecessors(graph)
    predec = zeros(Int, n)

    for iteration in 1:n
        updated = false
        for i in 1:n
            for k in predecs[i]
                new_distance = distances[k] + graph[k, i]
                if new_distance < distances[i]
                    distances[i] = new_distance
                    predec[i] = k
                    updated = true
                end
            end
        end
        if !updated
            break
        end
    end

    return distances, predec
end


bellmanFord (generic function with 1 method)

In [6]:
function findShortestPath(graph, source, destination)
    distances, predecessors = bellmanFord(graph, source)
    path = []

    # Construire le chemin en remontant depuis la destination jusqu'à la source
    current_vertex = destination
    while current_vertex != source
        push!(path, current_vertex)
        current_vertex = predecessors[convert(Int, current_vertex)]
    end
    push!(path, source)

    # Inverser le chemin pour l'obtenir dans le bon ordre, de la source à la destination
    return reverse(path)
end


findShortestPath (generic function with 1 method)

In [11]:
graph = [0 3 0 0 5 0;
         0 0 4 0 0 0;
         0 0 0 2 0 0;
         0 0 0 0 0 3;
         0 -1 0 9 0 0;
         0 0 0 0 0 0]


 # Graphe 1
graph1 = [0 2 0 6 0;
2 0 3 8 5;
0 3 0 0 7;
6 8 0 0 9;
0 5 7 9 0]

# Graphe 2
graph2 = [0 4 0 0;
0 0 5 0;
0 0 0 3;
0 0 0 0]

# Graphe 3
graph3 = [0 1 4 0 0;
0 0 0 2 0;
0 0 0 3 0;
0 0 0 0 1;
0 0 0 0 0]


# Fonction pour tester l'algorithme sur un graphe
function testBellmanFord(graph, source)
        distances, predecessors = bellmanFord(graph, source)
        println("Prédécesseurs : ", predecessors)
        println("\nDernière ligne contenant les coûts : ", distances)
        # Le sommet de destination pour le calcul du coût est le dernier sommet du graphe
        destination = size(graph, 1)
        println("\nCoût du chemin vers le sommet ", destination, " : ", distances[destination])
        println("----------------------------------")
    end
    
    # Test sur chaque graphe
    println("Test avec graphe 1")
    testBellmanFord(graph1, 1)
    
    println("Test avec graphe 2")
    testBellmanFord(graph2, 1)
    
    println("Test avec graphe 3")
    testBellmanFord(graph3, 1)

# Trouver et afficher les chemins
function printPath(graph, source, destination)
    path = findShortestPath(graph, source, destination)
    println("Chemin de ", source, " vers ", destination, " : ", path)
end

printPath(graph, source, 5)
printPath(graph, source, 6)
printPath(graph, 2, 4)
printPath(graph1, source, 5)
printPath(graph2, source, 4)
printPath(graph3, source, 3)

Test avec graphe 1
Prédécesseurs : [0, 1, 2, 1, 2]

Dernière ligne contenant les coûts : [0.0, 2.0, 5.0, 6.0, 7.0]

Coût du chemin vers le sommet 5 : 7.0
----------------------------------
Test avec graphe 2
Prédécesseurs : [0, 1, 2, 3]

Dernière ligne contenant les coûts : [0.0, 4.0, 9.0, 12.0]

Coût du chemin vers le sommet 4 : 12.0
----------------------------------
Test avec graphe 3
Prédécesseurs : [0, 1, 1, 2, 4]

Dernière ligne contenant les coûts : [0.0, 1.0, 4.0, 3.0, 4.0]

Coût du chemin vers le sommet 5 : 4.0
----------------------------------
Chemin de 1 vers 5 : Any[1, 5]
Chemin de 1 vers 6 : Any[1, 2, 3, 4, 6]
Chemin de 2 vers 4 : Any[2, 3, 4]
Chemin de 1 vers 5 : Any[1, 2, 5]
Chemin de 1 vers 4 : Any[1, 2, 3, 4]
Chemin de 1 vers 3 : Any[1, 3]


# 1-Programmation dynamique(Explication)


La programmation dynamique est une technique algorithmique utilisée pour résoudre des problèmes complexes en les décomposant en sous-problèmes plus petits et en résolvant ces sous-problèmes une seule fois. Elle est particulièrement utile pour des problèmes où des sous-problèmes similaires se répètent plusieurs fois. En stockant les solutions des sous-problèmes, la programmation dynamique évite des calculs redondants, ce qui rend l'algorithme plus efficace.

La relation de récurrence est l'élément clé de la programmation dynamique. Elle définit comment la solution d'un problème peut être exprimée en fonction des solutions de ses sous-problèmes. Par exemple, dans le problème du sac à dos, la relation de récurrence détermine la valeur maximale que l'on peut obtenir avec un ensemble d'objets et une capacité de sac donnée, en choisissant d'inclure ou non chaque objet. Cette approche assure que la solution optimale est construite à partir des solutions optimales des sous-problèmes.

# 2-Algorithme de programmation dynamique (Fonctionnement)

# Fonction calculate_max_value
Cette fonction calcule la valeur maximale pouvant être obtenue en considérant un objet spécifique à un indice donné (item_index) et une capacité actuelle du sac à dos (current_capacity). Elle utilise une table de programmation dynamique (dp_table) pour stocker et récupérer des valeurs calculées précédemment.

Cas de base : Si la current_capacity est 0, cela signifie qu'aucun objet ne peut être ajouté au sac à dos. Par conséquent, la fonction renvoie 0.
Poids Supérieur à la Capacité : Si le poids de l'objet actuel (item_weights[item_index]) est supérieur à la capacité actuelle du sac à dos, l'objet ne peut pas être ajouté. La fonction renvoie alors la valeur maximale obtenue sans cet objet, qui est déjà stockée dans dp_table.
Calcul de la Valeur Maximale : Si l'objet peut potentiellement être ajouté au sac à dos, deux scénarios sont envisagés :
Valeur sans l'objet actuel (value_without_item) : C'est la valeur maximale obtenue sans ajouter l'objet actuel au sac à dos.
Valeur avec l'objet actuel (value_with_item) : C'est la valeur de l'objet actuel ajoutée à la valeur maximale obtenue avec le reste de la capacité du sac à dos après avoir ajouté l'objet.
La fonction choisit la valeur maximale entre ces deux scénarios et la renvoie.
# Fonction knapsack
Cette fonction utilise calculate_max_value pour résoudre le problème du sac à dos pour un ensemble d'objets, chacun ayant une valeur et un poids, et pour une capacité maximale du sac à dos (max_capacity).

Initialisation de la Table de Programmation Dynamique : Une table dp_table est initialisée avec des zéros pour stocker les valeurs maximales pour différentes combinaisons de nombres d'objets et de capacités de sac à dos.
Remplissage de la Table : Le code itère sur chaque objet (item_index) et chaque capacité possible du sac à dos (current_capacity). À chaque itération, il appelle calculate_max_value pour obtenir la valeur maximale pour la combinaison actuelle et stocke cette valeur dans dp_table.
Récupération de la Valeur Maximale : Une fois la table entièrement remplie, la valeur en dp_table[num_items, max_capacity] représente la valeur maximale que l'on peut obtenir avec tous les objets disponibles et la capacité maximale du sac à dos. Cette valeur est renvoyée en tant que résultat final de la fonction knapsack.